> **NOTE**: you should run `openvino_benchmark.ipynb` notebook first that will generate `openvino_fp32` folder with the exported models that are used for quantization.

In [ ]:
import os
import torch

from PIL import Image
import open_clip

In [ ]:
name = "ViT-B-16-plus-240"
pretrained = "laion400m_e32"
model, train_transform, eval_transform = open_clip.create_model_and_transforms(name, pretrained=pretrained)

In [ ]:
from pathlib import Path

source_dir = "openvino_fp32"
ouptut_dir = "openvino_int8"
if not os.path.exists(ouptut_dir):
    os.makedirs(ouptut_dir)
with open(Path(ouptut_dir) / "model_index.txt", 'w') as fd:
    fd.write(f"{name},{pretrained}\n")

In [ ]:
tokenizer = open_clip.get_tokenizer('ViT-B-16-plus-240')

image = eval_transform(Image.open("../../docs/CLIP.png")).unsqueeze(0)
text = tokenizer(["a diagram", "a dog", "a cat"])

### Prepare data for optimization

In [ ]:
import random
from io import BytesIO
import requests
import numpy as np

def get_pil_from_url(url):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    return image.convert("RGB")

def check_text_data(data):
    if isinstance(data, str):
        return True
    if isinstance(data, list):
        return all(isinstance(x, str) for x in data)
    return False    

def laion2B_preprocess_train(examples, train_transforms, tokenize_captions, image_column="url", text_column="caption"):
    url = examples[image_column]
    try:
        image = get_pil_from_url(url)
        if not check_text_data(examples[text_column]):
            raise ValueError("Text data is not valid")
    except Exception:
        print(f"Can't load image from url: {url}")
        return None

    examples["pixel_values"] = train_transforms(image)
    examples["text"] = tokenize_captions(examples)
    return examples

def tokenize_captions(examples, is_train=True):
    caption_column = "caption"
    captions = []
    caption = examples[caption_column]
    if isinstance(caption, str):
        captions.append(caption)
    elif isinstance(caption, (list, np.ndarray)):
        # take a random caption if there are multiple
        captions.append(random.choice(caption) if is_train else caption[0])
    else:
        raise ValueError(f"Caption column `{caption_column}` should contain either strings or lists of strings.")
    input_ids = tokenizer(captions[0])[0]
    return input_ids

In [ ]:
from datasets import load_dataset

max_train_samples = 10000
dataset = load_dataset("laion/laion400m", streaming=True)
train_dataset = dataset["train"].shuffle(seed=42, buffer_size=max_train_samples)

In [ ]:
cast_dtype = model.transformer.get_cast_dtype()

def collate_fn(examples):
    examples = [laion2B_preprocess_train(example, train_transform, tokenize_captions) for example in examples]
    if not any(examples):
        return None
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()
    
    input_ids = torch.stack([example["text"] for example in examples])
    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids
    }

In [ ]:
import itertools
from tqdm.notebook import tqdm

def prepare_calibration_data(dataloader, init_steps):
    data = []

    print(f"Fetching {init_steps} for the initialization...")
    for _, batch in tqdm(zip(range(init_steps), itertools.islice(dataloader, 0, init_steps))):
        with torch.no_grad():
            # Convert images to latent space
            if batch:
                data.append(
                    (
                        batch["pixel_values"].to("cpu"),
                        batch["input_ids"].to("cpu")
                    )
                )
    return data

In [ ]:
batch_size = 1
dataloader_num_workers = 4
dataloader = torch.utils.data.DataLoader(
        train_dataset, collate_fn=collate_fn, batch_size=batch_size, num_workers=dataloader_num_workers
    )

In [ ]:
opt_init_steps = 500
calibration_data = prepare_calibration_data(dataloader, opt_init_steps)

### Quantize Image Encoder

In [ ]:
import nncf
from nncf.quantization.advanced_parameters  import AdvancedQuantizationParameters, AdvancedBiasCorrectionParameters
from nncf.scopes import IgnoredScope

ignored_scope = IgnoredScope(
    types = ["Divide"]
)


advanced_parameters = AdvancedQuantizationParameters(
    backend_params = {'use_pot': True}, 
    bias_correction_params = AdvancedBiasCorrectionParameters(apply_for_all_nodes=True, threshold=float('inf')),
    overflow_fix="disable")

def quantize_image_encoder(model, data_loader):
    quantization_dataset = nncf.Dataset(data_loader, lambda x: x[0])

    quantized_model = nncf.quantize(
                            model,
                            quantization_dataset,
                            model_type=nncf.ModelType.TRANSFORMER,
                            preset=nncf.QuantizationPreset.MIXED,
                            fast_bias_correction=False,
                            subset_size=opt_init_steps,
                            advanced_parameters=advanced_parameters,
                            ignored_scope=ignored_scope,
                            )
    return quantized_model

In [ ]:
import openvino.runtime as ov
from pathlib import Path

ov_model_path = Path(source_dir) / "image_encoder.xml"

core = ov.Core()
image_encoder = core.read_model(ov_model_path)

q_image_encoder = quantize_image_encoder(image_encoder, calibration_data)

In [ ]:
ov.serialize(q_image_encoder, ouptut_dir + "/image_encoder.xml")

### Quantize Text Encoder

In [ ]:
advanced_parameters = AdvancedQuantizationParameters(
    backend_params = {'use_pot': True}, 
    bias_correction_params = AdvancedBiasCorrectionParameters(apply_for_all_nodes=True, threshold=float('inf')),
    overflow_fix="disable"
    )


def quantize_text_encoder(model, data_loader):
    quantization_dataset = nncf.Dataset(data_loader, lambda x: x[1])

    quantized_model = nncf.quantize(
                            model,
                            quantization_dataset,
                            model_type=nncf.ModelType.TRANSFORMER,
                            preset=nncf.QuantizationPreset.MIXED,
                            fast_bias_correction=False,
                            subset_size=opt_init_steps,
                            advanced_parameters=advanced_parameters,
                            )
    return quantized_model

In [ ]:
ov_model_path = Path(source_dir) / "text_encoder.xml"

core = ov.Core()
text_encoder = core.read_model(ov_model_path)

q_text_encoder = quantize_text_encoder(text_encoder, calibration_data)

In [ ]:
ov.serialize(q_text_encoder, ouptut_dir + "/text_encoder.xml")